In [1]:
import os 


import pandas as pd
import numpy as np
import yfinance as yf


WORKSPACE_DIR = os.getenv('WORKSPACE_DIR')
if not os.getcwd().endswith('portfolio_py'):
    os.chdir(f'{WORKSPACE_DIR}/portfolio_py')
print(f'Current Working Directory: {os.getcwd()}')


from utils.config import PROGRAM_START_DATE, PROGRAM_END_DATE

Current Working Directory: /Users/blakeuribe/Desktop/portfolio_py
Updated on 04/15/2025 5:56


In [2]:
filtered_df = pd.read_csv('data/clean/filtered_valuation_df.csv')

tickers = list(filtered_df['Tickers'])

Get data

In [3]:
# start_date = '2024-06-15'
# end_date = '2025-06-15'

stock_data = yf.download(tickers, start=PROGRAM_START_DATE, end=PROGRAM_END_DATE)["Close"]
stock_data = stock_data.pct_change().dropna()

tbill_data = yf.download('^IRX', start=PROGRAM_START_DATE, end=PROGRAM_END_DATE, auto_adjust=True)['Close']
tbill_data = tbill_data / 100 / 360  # Convert to daily rate
tbill_data = tbill_data.iloc[1:] # to match the dimension of other stock data as we drop na due to pct


spy_data  = yf.download('SPY', start=PROGRAM_START_DATE, end=PROGRAM_END_DATE)["Close"]
spy_data = spy_data.pct_change().dropna()

# stock_data.head()
spy_data

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Ticker,SPY
Date,
2024-04-16,-0.001824
2024-04-17,-0.005918
2024-04-18,-0.002058
2024-04-19,-0.008728
2024-04-22,0.009209
...,...
2025-04-08,-0.015663
2025-04-09,0.105019
2025-04-10,-0.043819


In [4]:
excess_returns = stock_data.subtract(tbill_data['^IRX'], axis=0)
excess_returns

Ticker,ABT,AEM,FTNT,GRMN,GS,MMM,MPLX,MS,NEM,TMUS
Date,,,,,,,,,,
2024-04-16,0.002793,0.004901,-0.004008,0.003738,-0.010173,-0.002884,-0.005163,0.024570,-0.008730,-0.000333
2024-04-17,-0.030454,0.012002,0.002646,-0.008022,0.017619,-0.001134,0.008175,0.010400,0.011662,0.001732
2024-04-18,-0.006095,0.015537,-0.009889,-0.002627,-0.002126,0.005571,0.005356,0.001853,-0.000405,0.004602
2024-04-19,0.018948,0.005684,-0.009985,-0.008673,0.002062,0.008490,0.011543,0.004286,0.012046,0.009055
2024-04-22,-0.002103,-0.024895,0.004428,0.010605,0.032899,0.003648,0.012391,0.015627,-0.040125,0.000039
...,...,...,...,...,...,...,...,...,...,...
2025-04-08,-0.007404,0.004903,0.001589,-0.040092,-0.007184,-0.010929,-0.011552,-0.006160,0.002113,-0.001616
2025-04-09,0.024812,0.056509,0.133518,0.109771,0.118116,0.087646,0.033944,0.113431,0.084180,0.034799
2025-04-10,-0.020110,0.054231,-0.036242,-0.030629,-0.052490,-0.038795,-0.017933,-0.045954,0.044806,0.004670


Need to do poly fit

In [5]:
excess_returns['ABT'].values

array([ 2.79340326e-03, -3.04540686e-02, -6.09475620e-03,  1.89481342e-02,
       -2.10317637e-03,  4.71094500e-03, -6.65220797e-03, -4.26292621e-04,
        6.12435959e-03, -2.56357059e-03, -1.22647283e-02,  2.87398215e-03,
       -3.62657600e-03, -3.34170748e-04, -2.60077715e-03,  4.87140690e-03,
       -1.17308773e-02, -2.71845918e-03,  5.23075806e-04,  1.40876786e-04,
       -7.39932754e-03,  5.43085276e-03,  2.53141455e-03, -7.58326733e-03,
       -8.59981271e-03, -2.56787579e-03,  1.79196759e-02, -5.77434111e-03,
       -2.83221803e-03, -1.93857901e-02, -1.21123174e-02,  9.88086976e-03,
        4.27740419e-03,  6.50860832e-03,  5.29814867e-03, -2.07913952e-03,
        9.92916179e-03,  3.15991646e-02, -9.82045771e-04, -1.57751007e-02,
       -1.36604037e-02, -9.05535839e-03,  2.07780914e-03, -2.36407119e-03,
        5.07450648e-03,  2.46650286e-02, -8.12086490e-03, -4.11745518e-03,
        1.65919267e-03, -5.92784675e-03, -2.33791071e-03, -7.21606315e-03,
       -6.97776568e-03, -

In [6]:
polyfits = {} # reads as ticker: [Beta, alpha, residual]

# Loop over each column and fit a polynomial
for col in excess_returns.columns:
    y = excess_returns[col].values
    coeffs = np.polyfit(np.array(spy_data['SPY']), y, 1, full=True)  # returns highest degree first
    polyfits[col] = coeffs[:2]

In [7]:
polyfits

{'ABT': (array([0.23201908, 0.00057024]), array([0.0399995])),
 'AEM': (array([0.57602072, 0.0025575 ]), array([0.08975816])),
 'FTNT': (array([0.99125449, 0.00142318]), array([0.13626621])),
 'GRMN': (array([1.00042393, 0.00108378]), array([0.13025021])),
 'GS': (array([1.32781656e+00, 5.75742959e-04]), array([0.04518196])),
 'MMM': (array([0.88833471, 0.00143081]), array([0.08798373])),
 'MPLX': (array([0.51617968, 0.00093644]), array([0.02588999])),
 'MS': (array([1.28847372e+00, 6.22829306e-04]), array([0.04726513])),
 'NEM': (array([0.71304413, 0.0013905 ]), array([0.12421643])),
 'TMUS': (array([0.32985076, 0.001892  ]), array([0.04852548]))}

In [8]:
from utils.finance_utils import plot_cum_ret




---------------------------------
finance_utils.py successfully loaded, updated last April. 29 2025 4:55
---------------------------------




In [9]:
spy_data

Ticker,SPY
Date,
2024-04-16,-0.001824
2024-04-17,-0.005918
2024-04-18,-0.002058
2024-04-19,-0.008728
2024-04-22,0.009209
...,...
2025-04-08,-0.015663
2025-04-09,0.105019
2025-04-10,-0.043819


In [10]:
stock_data

Ticker,ABT,AEM,FTNT,GRMN,GS,MMM,MPLX,MS,NEM,TMUS
Date,,,,,,,,,,
2024-04-16,0.002939,0.005046,-0.003862,0.003883,-0.010028,-0.002738,-0.005018,0.024716,-0.008585,-0.000188
2024-04-17,-0.030309,0.012148,0.002792,-0.007877,0.017764,-0.000989,0.008321,0.010545,0.011808,0.001877
2024-04-18,-0.005949,0.015683,-0.009743,-0.002481,-0.001981,0.005717,0.005501,0.001998,-0.000259,0.004747
2024-04-19,0.019094,0.005829,-0.009839,-0.008527,0.002208,0.008636,0.011689,0.004432,0.012192,0.009201
2024-04-22,-0.001957,-0.024749,0.004574,0.010751,0.033045,0.003793,0.012537,0.015773,-0.039979,0.000185
...,...,...,...,...,...,...,...,...,...,...
2025-04-08,-0.007288,0.005019,0.001705,-0.039976,-0.007068,-0.010813,-0.011436,-0.006044,0.002229,-0.001500
2025-04-09,0.024929,0.056626,0.133636,0.109889,0.118234,0.087763,0.034062,0.113548,0.084297,0.034917
2025-04-10,-0.019994,0.054348,-0.036125,-0.030512,-0.052373,-0.038678,-0.017816,-0.045837,0.044923,0.004786
